In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [18]:
class Simulator:
    def __init__(
        self, 
        num_genes = 20000, 
        avg_num_sgRNAs = 5,  
        num_control = 2, 
        num_treatment = 2,
        min_total = 1000,
        max_total = 100000,
        total_NTCs = 1000,
        fraction_enriched = 0.2,
        fraction_depleted = 0.2,
        fraction_NTC = 0.2,
        type_dist = 2):
        
        """
        Constructor for initializing Simulator object.
        
        Parameters
        ----------
        num_genes : int
            Number of genes.
        avg_num_sgRNAs : int
            Average number of sgRNAs across all genes. 
        num_treatment : int
            Number of treatment of libraries.
        num_control : int
            Number of control of libraries. 
        min_total : int
            The lower bound of the total number of counts for one library. 
        max_total : int
            The upper bound of the total number of counts for one library.
        total_NTCs : int
            Total number of non-targeting controls.
        fraction_enriched : float
            The fraction of enriched genes with respect to all genes. 
        fraction_depleted : float
            The fraction of depleted genes with respect to all genes. 
        fraction_NTC : float
            The fraction of NTC genes with respect to all genes.
        type_dist : int
            1 is poisson distrubution and 2 is negative binomial distrubution. 
            
        Raises
        ------
        Exception
            If the total of fractions (enriched, depleted, NTC) exceeds 1. 
        
        """
        self.num_genes = num_genes
        self.avg_num_sgRNAs = avg_num_sgRNAs
        self.num_control = num_control
        self.num_treatment = num_treatment
        self.min_total = min_total
        self.max_total = max_total
        self.total_NTCs = total_NTCs
        self.type_dist = type_dist
        self.bounds = [10, 30]
        
        self._init_count_totals()
        self._init_fractions(fraction_enriched, fraction_depleted, fraction_NTC)
        self._init_num_sgRNAs()
        self._split_genes()
        self._init_lambda()
        self._init_p()
        self._init_S()
        

        
    def _init_count_totals(self):
        self.totals_array = np.random.randint(self.min_total, self.max_total, size = self.num_treatment + self.num_control)
        
    def _init_fractions(self, e, d, ntc):
        total = e + d + ntc
        
        if ((total > 0.0) & (total <= 1.0)):
            self.fraction_enriched = e
            self.fraction_depleted = d
            self.fraction_NTC = ntc
            self.fraction_normal = 1.0 - (e + d + ntc)
        else:
            raise Exception("Fractions total cannot exceed 1.") 
    
    def _init_num_sgRNAs(self):
        """
        Generates a number of sgRNAs per gene. 
        
        Returns
        -------
        sgRNAs : array
            The values of the array follow a normal distrubution with the
            mean being `avg_num_sgRNAs`.
        
        """
        sgRNAs = np.random.normal(loc=self.avg_num_sgRNAs, scale=1, size=self.num_genes)
        sgRNAs = np.round(sgRNAs)
        self.sgRNAs = sgRNAs 
        
    def _split_genes(self):
        num_e = round(len(self.sgRNAs) * self.fraction_enriched)
        num_d = round(len(self.sgRNAs) * self.fraction_depleted)
        num_ntc = round(len(self.sgRNAs) * self.fraction_NTC)
        num_n = round(len(self.sgRNAs) * self.fraction_normal)
        
        self.g_e = self.sgRNAs[0: num_e]
        self.g_d = self.sgRNAs[num_e: num_e + num_d]
        self.g_ntc = self.sgRNAs[num_e + num_d: num_e + num_d + num_ntc]
        self.g_n = self.sgRNAs[num_e + num_d + num_ntc: num_e + num_d + num_ntc + num_n]
        
    def _init_lambda(self):
        self.lam = np.random.uniform(self.bounds[0], self.bounds[1], size = int(self.sgRNAs.sum()))

    def _init_p(self):
        self.p = np.random.random(size = int(self.sgRNAs.sum()))
    
    def _init_S(self):
        S = []

        for i in self.g_e:
            g_scalar = np.random.uniform(1.2, 2.0)
            for n in np.arange(i):
                S.append(g_scalar)

        for i in self.g_d:
            g_scalar = np.random.uniform(0.2, 1.0)
            for n in np.arange(i):
                S.append(g_scalar)
                
        for i in self.g_ntc:
            for n in np.arange(i):
                S.append(1)
                
        for i in self.g_n:
            for n in np.arange(i):
                S.append(1)
            
        self.S = S 
    
    def _sgRNAs(self):
        return ["sgRNA_" + str(int(i)) for i in np.arange(self.sgRNAs.sum())]
    
    def _gene(self):
        """
        Generates list of numbered genes for use in sample() DataFrame. 
        
        Returns
        -------
        list
            The elements of the list are in numerical order up to the 
            number of genes in `num_genes`.
        
        """
        return ["gene_" + str(i) for i in np.arange(len(self.sgRNAs)) for n in np.arange(self.sgRNAs[i])]
    
        
    def _sum_array(self, index, lambdas, p_array):
        """
        Creates an array of random integers with a specified sum.
        
        Parameters
        ----------
        index : int
            The index to specify which total to use from `totals_array` 
            defined in the constructor. 
            
        Returns
        -------
        a : array
            array of randomly generated integers with sum of element from `totals_array`    
        
        """
        
        if self.type_dist == 1:
            a = [np.random.poisson(i, size=1) for i in lambdas]
        elif self.type_dist == 2:
            a = [np.random.negative_binomial(i, p, size=1) for i in lambdas for p in p_array]
        else:
            raise Exception("Make sure to choose a type from the available ints")
        
        a = np.concatenate(a)
        a = a.astype(float)
        a /= (a.sum())
        a *= self.totals_array[index]
        a = np.round(a)
        
        return a
    
    def _setting_treatment_libraries(self):
        """
        Generates values for treatment libraries.
        
        Returns
        -------
        treatment : list
            `treatment` is a list of arrays, one for each library, 
            generated by the _sum_array() method. 
            
        """
        treatment = [] 
        
        for i in np.arange(self.num_treatment):
            treatment.append(self._sum_array(i, self._S_l(), self.p))
        
        return treatment
    
    def _setting_control_libraries(self):
        """
        Generates values for control libraries.
        
        Returns
        -------
        control : list
            `control` is a list of arrays, one for each library, 
            generated by the _sum_array() method. 
            
        """
        control = [] 
        
        for i in np.arange(self.num_control):
            control.append(self._sum_array(-(i+1), self.lam, self.p))
        
        return control
    
    def _S_l(self):
        return np.multiply(self.S, self.lam)
             
    def _type_of_change(self):
        """
        Sets number of enriched, depleted, NTC, and normal genes. 
        
        Returns
        -------
        type_of_change : list
            The list is filled with strings of enriched, depleted, NTC, 
            and normal for a gene. The number of times a type is in the 
            list is based on the fractional representation specified upon 
            initialization.
            
        """
        
        type_of_change = []
        
        e = ["enriched" for i in np.arange(len(self.g_e)) for n in np.arange(self.g_e[i])]
        d = ["depleted" for i in np.arange(len(self.g_d)) for n in np.arange(self.g_d[i])]
        ntc = ["ntc" for i in np.arange(len(self.g_ntc)) for n in np.arange(self.g_ntc[i])]
        n = ["normal" for i in np.arange(len(self.g_n)) for n in np.arange(self.g_n[i])]
        
        type_of_change = e + d + ntc + n
        
        return type_of_change 
    
    def sample(self, seed = 10):
        """
        Generates DataFrame with observations for the simulation. 
        
        Returns
        -------
        result : DataFrame 
            This DataFrame concatenates information from the _gene(),
            _num_sgRNAs(), _setting_treatment_libraries(),
            _setting_control_libraries(), _type_of_change() methods.  
            
        """
        
        # reorganize this to make code clearer
        
        np.random.seed(seed)
        # currently, every instance is initialized with lambda, sgRNA numbers, count totals, so 
        # they are the same regardless of sample(). 
        # the seed only keeps the setting libraries the same each time sample is called on the same object
        # should that change so there is a seed for everything? 
    
        sgRNA = pd.DataFrame({"sgRNAs": self._sgRNAs()})
        gene = pd.DataFrame({"gene": self._gene()})
        lam = pd.DataFrame({"lambda": self.lam})
        S_lam = pd.DataFrame({"modified lambda": self._S_l()})
        control = pd.DataFrame(self._setting_control_libraries()).T
        treatment = pd.DataFrame(self._setting_treatment_libraries()).T
        type_of_change = pd.DataFrame({"type": self._type_of_change()})
        
        result = pd.concat([sgRNA, gene, lam, S_lam, control, treatment, type_of_change], axis=1, join="inner")

        return result 

In [33]:
trial = Simulator(num_genes=5, num_control=1, num_treatment=1, fraction_depleted=0.2, type_dist = 1)

In [34]:
print(trial.sample(4))

      sgRNAs    gene     lambda  modified lambda      0       0      type
0    sgRNA_0  gene_0  11.495275        15.877421  336.0   748.0  enriched
1    sgRNA_1  gene_0  10.613035        14.658860  528.0   442.0  enriched
2    sgRNA_2  gene_0  24.759496        34.198132  504.0  1224.0  enriched
3    sgRNA_3  gene_0  29.939257        41.352484  841.0  1462.0  enriched
4    sgRNA_4  gene_0  14.262047        19.698921  240.0   680.0  enriched
5    sgRNA_5  gene_0  12.922111        17.848185  216.0   340.0  enriched
6    sgRNA_6  gene_1  18.782730        15.736195  480.0   340.0  depleted
7    sgRNA_7  gene_1  24.875871        20.841037  672.0   646.0  depleted
8    sgRNA_8  gene_1  23.081777        19.337942  552.0   306.0  depleted
9    sgRNA_9  gene_1  16.837087        14.106133  408.0   544.0  depleted
10  sgRNA_10  gene_2  12.159564        12.159564  192.0   442.0       ntc
11  sgRNA_11  gene_2  18.314989        18.314989  456.0   612.0       ntc
12  sgRNA_12  gene_2  20.954574       